# WEEK 1: The Battle of Neighborhoods

## PART 1: Introduction 

#### 1. Importing important libraries

In [2]:
import numpy as np 
import time
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
import requests 
from pandas.io.json import json_normalize 

from geopy.geocoders import Nominatim 


print('Libraries imported.')

Libraries imported.


### Scenario:
I am a data scientist residing in Downtown Singapore. I currently live within walking distance to Downtown Telok Ayer MRT metro station and I enjoy many ammenities and venues in the area, such as various international cousine restaurants, cafes, food shops and entertainment. I have been offered a great opportunity to work for a leader firm in Manhattan, NY. I am very excited and I want to use this opportunity to practice my learnings in Coursera in order to answer relevant questions arisen. The key question is : How can I find a convenient and enjoyable place similar to mine now in Singapore? Certainly, I can use available real estate apps and Google but the idea is to use and apply myself the learned tools during the course. In order to make a comparison and evaluation of the rental options in Manhattan NY, I must set some basis, therefore the apartment in Manhattan must meet the following demands:

-apartment must be 2 or 3 bedrooms, desired location is near a metro station in the Manhattan area and within 1.0 mile (1.6 km) radius, price of rent not exceed $7,000 per month, top ammenities in the selected neighborhood shall be similar to current residence and desirable to have venues such as coffee shops, restaurants Asian Thai, wine stores, gym and food shops. 

### Business Problem:
The challenge is to find a suitable apartment for rent in Manhattan NY that complies with the demands on location, price and venues. The data required to resolve this challenge is described in the following section 2, below.

### Interested Audience
I believe this is a relevant challenge with valid questions for anyone moving to other large city in US, EU or Asia. The same methodology can be applied in accordance to demands as applicable. This case is also applicable for anyone interested in exploring starting or locating a new business in any city. Lastly, it can also serve as a good practical exercise to develop Data Science skills.

## PART 2: Data sections

#### Description of the Data:
###### The following data is required to answer the issues of the problem:
List of Boroughs and neighborhoods of Manhattan with their geodata (latitud and longitud)
List of Subway metro stations in Manhattan with their address location
List of apartments for rent in Manhattan area with their addresses and price
Preferably, a list of apartment for rent with additional information, such as price, address, area, # of beds, etc
Venues for each Manhattan neighborhood ( than can be clustered)
Venues for subway metro stations, as needed

#### How the data will be used to solve the problem
###### The data will be used as follows:
Use Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups ( as per Course LAB)
Use foursquare and geopy data to map the location of subway metro stations , separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each metro station, or explore each subway location separately
Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway locations.
create a map that depicts, for instance, the average rental price per square ft, around a radious of 1.0 mile (1.6 km) around each subway station - or a similar metrics. I will be able to quickly point to the popups to know the relative price per subway area.
Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.
Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies such as Metro New York MTA, etc.
The procesing of these DATA will allow to answer the key questions to make a decision:
what is the cost of rent (per square ft) around a mile radius from each subway metro station?
what is the area of Manhattan with best rental pricing that meets criteria established?
What is the distance from work place ( Park Ave and 53 rd St) and the tentative future home?
What are the venues of the two best places to live? How the prices compare?
How venues distribute among Manhattan neighborhoods and around metro stations?
Are there tradeoffs between size and price and location?
Any other interesting statistical data findings of the real estate and overall data.

Reference of venues around current residence in Singapore for comparison to Manhattan place:

In [3]:
# Shenton Way, District 01, Singapore
address = 'Mccallum Street, Singapore'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore home are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of Singapore home are 1.2784801, 103.8493717.


In [4]:
neighborhood_latitude=1.2792655
neighborhood_longitude=103.8480938

In [9]:

LIMIT = 100 
radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id=DVCTZDPDYXTS0BRJFPLMHM323APGXNWZI5PLRQ1VC0CFLF1T&client_secret=5NWAGXRLXIXAV0L3DNYY1EPIHNMAAAIZFDFELYSYXL5LFWL1&v=20180605&ll=1.2792655,103.8480938&radius=500&limit=100'
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=DVCTZDPDYXTS0BRJFPLMHM323APGXNWZI5PLRQ1VC0CFLF1T&client_secret=5NWAGXRLXIXAV0L3DNYY1EPIHNMAAAIZFDFELYSYXL5LFWL1&v=20180605&ll=1.2792655,103.8480938&radius=500&limit=100'

In [10]:
results = requests.get(url).json()

In [11]:
#Catogaries:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(10)

,name,categories,lat,lng
0,Napoleon Food & Wine Bar,Wine Bar,1.279925,103.847333
1,Native,Cocktail Bar,1.280135,103.846844
2,Freehouse,Beer Garden,1.281254,103.848513
3,Sofitel So Singapore,Hotel,1.280017,103.849813
4,Park Bench Deli,Deli / Bodega,1.279872,103.847287
5,Pepper Bowl,Asian Restaurant,1.279371,103.846710
6,Meat Smith,Southern / Soul Food Restaurant,1.280205,103.847410
7,Mellower Coffee,Café,1.277814,103.848188
8,PS.Cafe,Café,1.280468,103.846264
9,Anglo Indian Cafe & Bar,Indian Restaurant,1.279084,103.850127
